In [1]:
import numpy as np
import pandas as pd
import pickle
from scipy.sparse import csr_matrix

from models.group_recommender_mf import GroupRecommenderMF
from models.explicit_mf_with_bias import SGDExplicitBiasMF

In [2]:
# with open("../../data/model_sgd_mf_v4_50__1666837325.pkl", "rb") as f:
#     mf_full_model = pickle.load(file=f)
# mf_full_model.item_bias.shape
# mf_full_model.item_vecs.shape

In [3]:
group_mf = GroupRecommenderMF(
    full_model_file_path = "../../data/model_sgd_mf_v4_50__1666837325.pkl",
    item_encoder_file_path="../../data/anime_encoder.csv")

In [4]:
group_mf.item_vecs.shape

(5569, 128)

In [5]:
group_mf.item_encoder_df

,original_id,encoded_id
0,1,0
1,6,1
2,7,2
3,8,3
4,15,4
...,...,...
5564,45615,5564
5565,45616,5565
5566,46000,5566
5567,47398,5567


In [6]:
group_rating_df = pd.read_csv("../../data/group_rating_real.csv")
group_rating_df.columns = ["user_name", "item_id", "rating"]
group_rating_df

,user_name,item_id,rating
0,Kien,20,9
1,Kien,1735,8
2,Kien,918,10
3,Kien,21,6
4,Kien,30276,10
5,Kien,25777,10
6,Kien,40028,9
7,Ajay Friend,30276,10
8,Ajay Friend,20,9
9,Ajay Friend,1735,8


In [7]:
agg_df = group_rating_df.groupby("item_id")["rating"].mean()
agg_df.shape

(23,)

In [12]:
recommendations = group_mf.recommend_group(group_rating_df, reg=20)
recommendations

,item_id,rating
9,790,9.389130
8,32995,9.334661
7,38524,9.164376
6,35760,9.062798
5,9989,8.759514
4,2904,8.737653
3,30503,8.731807
2,6594,8.714520
1,22135,8.678085
0,37517,8.631050


In [13]:
anime_df = pd.read_csv("../../data/anime.csv")
recommendations.merge(anime_df, left_on="item_id", right_on="MAL_ID", how="inner")[
    ["item_id", "Name", "English name", "rating"]
]

,item_id,Name,English name,rating
0,790,Ergo Proxy,Ergo Proxy,9.389130
1,32995,Yuri!!! on Ice,Yuri!!! On ICE,9.334661
2,38524,Shingeki no Kyojin Season 3 Part 2,Attack on Titan Season 3 Part 2,9.164376
3,35760,Shingeki no Kyojin Season 3,Attack on Titan Season 3,9.062798
4,9989,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada...,anohana:The Flower We Saw That Day,8.759514
5,2904,Code Geass: Hangyaku no Lelouch R2,Code Geass:Lelouch of the Rebellion R2,8.737653
6,30503,Noragami Aragoto,Noragami Aragoto,8.731807
7,6594,Katanagatari,Katanagatari,8.714520
8,22135,Ping Pong the Animation,Ping Pong the Animation,8.678085
9,37517,Happy Sugar Life,Unknown,8.631050
